In [1]:
import cvxpy as cp

# 저자

- TA: 성준모 (Joonmo Sung)
- SYSTEMS MODELING AND PROGRAMMING LAB DEPARTMENT OF INDUSTRIAL ENGINEERING, YONSEI UNIVERSITY - SYMPLY
- 문의: `sjm21314@naver.com`,`sjm21314@yonsei.ac.kr`
***
본 강의자료는 Keha,A. B.,Khowala,K.,&Fowler,J. W. (2009). Mixed integer programming formulations for single machine scheduling problems. Computers & Industrial Engineering, 56(1), 357-367 논문을 바탕으로 제작되었으며,

반도체데이터사이언스 협동과정 수업 용도 이외의 목적으로 저자의 허락 없이 다른 사람들과 공유할 수 없습니다.

## Dataset
### Set
- N = $\{1,2,...,n\}, j \in N$
### Parameters
- $p_j$: processing time of job $j$
- $d_j$: due date of job $j$
- $w_j$: weight of job $j$
- $r_j$: release date of job $j$

In [4]:
# Offline 상황을 가정.

n = 5 # 작업의 개수

N = [_ for _ in range(n)] # 작업 set을 리스트 자료구조를 이용해 0, 1, 2,..., n-1 까지 담아줌
p = [4, 3, 7, 2, 2] # 가공시간 데이터를 리스트 자료구조를 이용해 만들어 줌
d = [5, 6, 8, 8, 17] # 납기 데이터를 리스트 자료구조를 이용해 만들어 줌
w = [5, 4, 3, 2, 1] # 가중치 데이터를 리스트 자료구조를 이용해 만들어 줌
r = [1, 2, 3, 3, 5] # 작업의 release date(도착 시점) 데이터를 리스트 자료구조를 이용해 만들어 줌

M = (sum(p) + sum(r)) *2 # big M

In [3]:
obj_select = 'twt' # 'wct', 'tj', 'twt', 'L_max' 중 하나 선택

## 3. Linear ordering variables

### Decision Variables
- $\delta_{j,k}$: 작업 $j$가 작업 $k$보다 앞에서 시작하면 1, 아니면 0인 이진변수, $\delta_{j,k} \in \{0, 1\} \ \forall j \in N, \forall k \in N$

In [5]:
# Decision Variable 만들기
delta = {(j, k): cp.Variable(boolean = True) for j in N for k in N}

### 기본 constraint
- $\delta_{j,k} + \delta_{k,j} = 1 \quad \forall j,k \in N, \ 1 \leq j < k \leq n,$, j<k 가 맞음. 등호 있을 시 infeasible
- $\delta_{j,k} + \delta_{k,l} + \delta_{l,j} \leq 2 \quad \forall j,k,l \in N, \ and \ j \neq k \neq l$

In [6]:
# 제약식 리스트 선언
constraints = []

# 제약식 추가
constraints += [delta[j,k] + delta[k,j] == 1 for j in N for k in N if j < k]
constraints += [delta[j,k] + delta[k,l] + delta[l,j] <= 2 for j in N for k in N for l in N if j != k and k != l and j != l]

#### To minimize the total weighted completion time
$ Minimize \sum_{j,k \in N, k \neq j}w_jp_j\delta_{k,j} + \sum_{j \in N}w_jp_j$

In [8]:
if obj_select == 'wct':
    # 목적식
    obj = cp.Minimize(sum(sum(w[j]*p[k]*delta[k,j] for j in N) for k in N if k != j) + sum(w[j]*p[j] for j in N))

#### To minimize $L_{max}$
$ Minimize \ L_{max}$

$ \\ s.t. \  L_{max} \geq (C_j - d_j) \quad \forall j \in N$

$ \\ C_j = \sum_{k \in N, k \neq j}p_k\delta_{k,j} + p_j \quad \forall j \in N$

In [11]:
if obj_select == "L_max":
    # Lmax 선언
    L_max = cp.Variable()

    # Lmax 제약식
    constraints += [L_max >= (sum(p[k]*delta[k,j] for k in N if k != j) + p[j] - d[j]) for j in N] # C_j 만들지 않고 바로 넣어줌

    # 목적식
    obj = cp.Minimize(L_max)

#### To minimize the number of tardy jobs

$ Minimize \ \sum_{j=1}^n U_j$

$ \\ s.t. \ C_j \leq d_j + MU_j \quad \forall j \in N$

$ \\ U_j \in \{0, 1\} \quad \forall j \in N$

$ \\ C_j = \sum_{k \in N, k \neq j}p_k\delta_{k,j} + p_j \quad \forall j \in N$

In [13]:
if obj_select == "tj":
    U = {(j): cp.Variable(boolean = True) for j in N} # j가 Tardy job이면 1, 아니면 0인 이진변수

    # Tardy job 제약식
    constraints += [sum(p[k]*delta[k,j] for k in N if k != j) + p[j] <= d[j] + M*U[j] for j in N]

    # 목적식
    obj = cp.Minimize(sum(U[j] for j in N))

#### To minimize the total weighted tardiness

$ Minimize \ \sum_{j=1}^n w_jT_j$

$ \\ s.t. \ T_j \geq C_j - d_j \quad \forall j \in N$

$ \\ T_j \geq 0 \quad \forall j \in N$

$ \\ C_j = \sum_{k \in N, k \neq j}p_k\delta_{k,j} + p_j \quad \forall j \in N$

In [14]:
if obj_select == "twt":
    T = {(j): cp.Variable(nonneg = True) for j in N} # j에 대한 Tardiness 연속형 변수, nonneg = True (0 이상)

    # Tardiness 제약식
    constraints += [T[j] >= sum(p[k]*delta[k,j] for k in N if k != j) + p[j] - d[j] for j in N] # C_j 없이 바로 넣어줌

    # 목적식
    obj = cp.Minimize(sum(w[j]*T[j] for j in N))

In [15]:
prob = cp.Problem(obj, constraints)
prob.solve()

np.float64(31.0)

In [16]:
print("최적해 상태:", prob.status)
print("최적값:", prob.value)

최적해 상태: optimal
최적값: 31.0


In [17]:
for (j, k), dv in delta.items():
    if dv.value:
        print(f"delta_{j+1}_{k+1}: {dv.value}")

delta_1_2: 1.0
delta_1_3: 1.0
delta_1_4: 1.0
delta_1_5: 1.0
delta_2_3: 1.0
delta_2_4: 1.0
delta_2_5: 1.0
delta_3_5: 1.0
delta_4_3: 1.0
delta_4_5: 1.0
